In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
from ax.modelbridge.factory import Models
import seabreeze.spectrometers as sb
from pathlib import Path
from colorama import Fore
import time
from scipy.interpolate import interp1d

print(Fore.GREEN + 'Successfully imported all the packages.')

obj1_name='peak_position'
obj2_name='intensity_ratio'

# Connect to the spectrophotometer
devices = sb.list_devices()
if not devices:
    print(Fore.RED + 'No Spectrophotometer Found!')
    # Uncomment if you want to exit during debugging
    # exit()
spec = sb.Spectrometer(devices[0])
print(Fore.GREEN + 'Spectrophotometer successfully connected.')
spec.integration_time_micros(40000)

# Initialize lists to store data for plotting
peak_position_values = []  # Updated name
intensity_ratio_values = []
iteration_numbers = []
parameter_history = []

# Define the updated objective function
def objective_function(processed_data, obj1_name='peak_position', obj2_name='intensity_ratio'):
    data_500_1100 = processed_data[(processed_data["Wavelength"] >= 500) & (processed_data["Wavelength"] <= 1100)]
    peak_intensity_row = data_500_1100.loc[data_500_1100["Normalized_Intensity"].idxmax()]
    peak_position = peak_intensity_row["Wavelength"]
    max_peak_intensity = peak_intensity_row["Normalized_Intensity"]

    # Find intensity at 400 nm
    intensity_at_400 = processed_data.loc[processed_data["Wavelength"] == 400, "Normalized_Intensity"]
    intensity_at_400 = intensity_at_400.values[0] if not intensity_at_400.empty else 0

    # Calculate intensity ratio
    ratio = max_peak_intensity / intensity_at_400 if intensity_at_400 != 0 else np.inf
    print(f"Calculated peak position: {peak_position} nm")
    print(f"Calculated intensity ratio: {ratio}")
    return {obj1_name: peak_position, obj2_name: ratio}

# Initialize the plots outside the loop
def initialize_plots():
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))  # 2 rows, 3 columns

    # Configure each plot's title, labels, and hide top/right spines
    for ax in axs.flat:
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

    axs[0, 0].set_title('Max Peak Position vs Iterations')
    axs[0, 0].set_xlabel('Iteration')
    axs[0, 0].set_ylabel('Max Peak Position')

    axs[0, 1].set_title('Intensity Ratio vs Iterations')
    axs[0, 1].set_xlabel('Iteration')
    axs[0, 1].set_ylabel('Intensity Ratio')

    axs[0, 2].set_title('Suggested Parameter x1 vs Iteration')
    axs[0, 2].set_xlabel('Iteration')
    axs[0, 2].set_ylabel('x1')

    axs[1, 0].set_title('Suggested Parameter x2 vs Iteration')
    axs[1, 0].set_xlabel('Iteration')
    axs[1, 0].set_ylabel('x2')

    axs[1, 1].set_title('Suggested Parameter x3 vs Iteration')
    axs[1, 1].set_xlabel('Iteration')
    axs[1, 1].set_ylabel('x3')

    axs[1, 2].set_title('Suggested Parameter x4 vs Iteration')
    axs[1, 2].set_xlabel('Iteration')
    axs[1, 2].set_ylabel('x4')

    plt.tight_layout()
    plt.ion()  # Enable interactive mode
    return fig, axs


def process_spectrum(df, start_wavelength=300, end_wavelength=1100, increment=1):
    # Step 2: Convert the columns to numeric (in case they are not) and handle any conversion errors
    df[0] = pd.to_numeric(df[0], errors='coerce')  # First column assumed to be wavelengths
    df[1] = pd.to_numeric(df[1], errors='coerce')  # Second column assumed to be intensities

    # Drop any rows with missing data (NaNs)
    df = df.dropna()

    # Step 3: Normalize the second column (intensity or absorbance values)
    df[1] = (df[1] - df[1].min()) / (df[1].max() - df[1].min())

    # Step 4: Resample the DataFrame to include only wavelengths between start and end, in increments
    new_wavelengths = np.arange(start_wavelength, end_wavelength + increment, increment)

    # Interpolate to fit the new wavelength range
    resampled_df = pd.DataFrame({'Wavelength': new_wavelengths})
    resampled_df['Normalized_Intensity'] = np.interp(new_wavelengths, df[0], df[1])

    return resampled_df

def update_plots(fig, axs, ref_df, new_spectrum_df, iteration_count, peak_position_values, intensity_ratio_values, parameter_history):
    params_array = np.array(parameter_history)

    for ax in axs.flat:
        ax.cla()  # Clear each axis for fresh plotting

    # Plot objective 1 (Max Peak Intensity) vs Iterations
    axs[0, 0].plot(iteration_numbers, peak_position_values, marker='o', color='green', label='Max Peak Intensity')
    axs[0, 0].set_title('Max Peak Intensity vs Iterations')
    axs[0, 0].set_xlabel('Iteration')
    axs[0, 0].set_ylabel('Max Peak Intensity')

    # Plot objective 2 (Intensity Ratio) vs Iterations
    axs[0, 1].plot(iteration_numbers, intensity_ratio_values, marker='o', color='blue', label='Intensity Ratio')
    axs[0, 1].set_title('Intensity Ratio vs Iterations')
    axs[0, 1].set_xlabel('Iteration')
    axs[0, 1].set_ylabel('Intensity Ratio')

    # Plot parameters x1, x2, x3, and x4 vs Iterations
    axs[0, 2].plot(iteration_numbers, params_array[:, 0], marker='o', color='orange', label='x1')
    axs[0, 2].set_xlabel('Iteration')
    axs[0, 2].set_ylabel('x1')

    axs[1, 0].plot(iteration_numbers, params_array[:, 1], marker='o', color='purple', label='x2')
    axs[1, 0].set_xlabel('Iteration')
    axs[1, 0].set_ylabel('x2')

    axs[1, 1].plot(iteration_numbers, params_array[:, 2], marker='o', color='red', label='x3')
    axs[1, 1].set_xlabel('Iteration')
    axs[1, 1].set_ylabel('x3')

    axs[1, 2].plot(iteration_numbers, params_array[:, 3], marker='o', color='brown', label='x4')
    axs[1, 2].set_xlabel('Iteration')
    axs[1, 2].set_ylabel('x4')

    fig.canvas.draw()
    plt.pause(0.5)

# Define paths
BASE_DIR_PATH = os.path.dirname(os.path.abspath(os.getcwd()))
DATA_UV_DIR_PATH = os.path.join(BASE_DIR_PATH, 'data', 'UV-Vis-NIR', '')
folder_path = DATA_UV_DIR_PATH
REF_SPECTRUM_DIR_PATH = os.path.join(BASE_DIR_PATH, 'data', 'reference_spectrum', '')
OUTPUT_DIR_PATH = os.path.join(BASE_DIR_PATH, 'hardware', 'PREST', 'src', '')
OUTPUT_DIR_PATH_2 = os.path.join(BASE_DIR_PATH, 'hardware', 'PREST_2', 'src', '')

# Load baseline data
baseline_data = pd.read_csv('spectrometer_data.txt', delimiter='\t')
#baseline_data['wavelength'] = baseline_data['wavelength'].astype(float)  # Ensure baseline wavelength is float

iteration_count = 0
num_iterations = 10

# Measure absorbance based on sample spectrum and baseline data
def measure_abs(sample_spectrum, iteration_count=0):
    divisor = sample_spectrum - baseline_data['dark']
    divisor[divisor <= 0] = np.nan  # Replace non-positive values to avoid log10 issues
    absorbance = np.log10((baseline_data['baseline'] - baseline_data['dark']) / divisor)
    absorbance = absorbance.replace([np.inf, -np.inf], np.nan).fillna(0)  # Replace NaNs and Infs

    absorbance_df = pd.DataFrame({
        'Wavelength': baseline_data['wavelength'],  # Ensure Wavelength is float
        'Absorbance': absorbance
    })
    
    filename = f"absorbance_{iteration_count}.txt"
    filepath = os.path.join(DATA_UV_DIR_PATH, filename)
    absorbance_df.to_csv(filepath, index=False)
    print(f"Absorbance data saved to {filepath}")
    
    return absorbance_df

# Function to write a specific set of values into suggested_parameters_2.txt
def write_suggested_parameters_2(x5,x6,x7,x8):
    param_sugg_2 = pd.DataFrame({
        'iteration': [iteration_count + 1],
        'x1': [x5],
        'x2': [x6],
        'x3': [x7],
        'x4': [x8]
    })
    write_header = not os.path.exists(parameters_2_txt_path)
    param_sugg_2.to_csv(parameters_2_txt_path, mode='a', header=write_header, index=False)
    print("Written to suggested_parameters_2.txt")

# Define the generation strategy
num_trials_SOBOL = 4
gs = GenerationStrategy(steps=[
    GenerationStep(model=Models.SOBOL, num_trials=num_trials_SOBOL, min_trials_observed=3, max_parallelism=1, model_kwargs={"seed": 987}),
    GenerationStep(model=Models.FULLYBAYESIAN, num_trials=-1, max_parallelism=1, model_kwargs={}),
])


# Initialize the Ax client with the generation strategy
ax_client = AxClient(generation_strategy=gs)

# Create the experiment
ax_client.create_experiment(parameters=[
    {"name": "x1", "type": "range", "bounds": [100, 1000]},
    {"name": "x2", "type": "range", "bounds": [100, 1000]},
    {"name": "x3", "type": "range", "bounds": [100, 1000]},
    {"name": "x4", "type": "range", "bounds": [100, 1000]},
], objectives={
        obj1_name: ObjectiveProperties(minimize=True),
        obj2_name: ObjectiveProperties(minimize=False),
    },
    tracking_metric_names=["max_peak_intensity", "intensity_ratio"], parameter_constraints=["x1 + x2 + x3 + x4 <= 2020" , "x1 + x2 + x3 + x4 >= 1990"])

# Define the text file path for saving suggested parameters
parameters_txt_path = os.path.join(OUTPUT_DIR_PATH, 'suggested_parameters.txt')
parameters_2_txt_path = os.path.join(OUTPUT_DIR_PATH_2, 'suggested_parameters_2.txt')

# Main optimization loop
spectra_files = []

ref_df = pd.read_csv(os.path.join(REF_SPECTRUM_DIR_PATH, 'ref_spectrum.txt'), skiprows=14, header=None, delimiter='\t')
ref_df = process_spectrum(ref_df)
print(Fore.RED + 'REF. SPECTRUM', ref_df)

fig, axs = initialize_plots()

known_files = set(os.listdir(DATA_UV_DIR_PATH))  # List of known files in the folder

folder_path = DATA_UV_DIR_PATH
# Check for new files
# Keep track of processed files
processed_files = set(os.listdir(folder_path))  # Files that are already processed

while iteration_count < num_iterations:
    parameterization, trial_index = ax_client.get_next_trial()
    x1, x2, x3, x4 = parameterization.get("x1"), parameterization.get("x2"), parameterization.get("x3"), parameterization.get("x4")

    print(f"Iteration {iteration_count + 1}: Suggested parameters - x1: {x1}, x2: {x2}, x3: {x3}, x4: {x4}")

    parameter_history.append([x1, x2, x3, x4])
    iteration_numbers.append(iteration_count + 1)

    param_sugg = pd.DataFrame({
        'iteration': [iteration_count + 1],
        'x1': [x1],
        'x2': [x2],
        'x3': [x3],
        'x4': [x4]
    })

    write_header = not os.path.exists(parameters_txt_path)
    param_sugg.to_csv(parameters_txt_path, mode='a', header=write_header, index=False)
    
    write_suggested_parameters_2(2000,2000,0,0)

    print(Fore.RED + 'Waiting for 1 hour for the reaction to complete...')
    time.sleep(10) # wait for 10 seconds for the reaction to complete

    write_suggested_parameters_2(0,0,5000,0)
    # TAKING THE SPECTRUM OF THE REACTION MIXTURE 

    print(Fore.RED + 'Taking the spectrum of the reaction mixture...')
    sample_spectrum = spec.intensities()
    measure_abs(sample_spectrum)
    sample_spectrum = process_spectrum(sample_spectrum)

    # CHECK THE ABOVE CODE BLOCK AND MAKE SURE THAT THE SAMPLE_SPECTRUM IS DEFINED AND WILL BE USED IN THE NEXT BLOCK.

    while True:
        files = os.listdir(folder_path)  # List of all files in the directory
            # Find new .txt files that haven't been processed
        new_files = [f for f in files if f.endswith(".txt") and f not in processed_files]
            
        if new_files:
                # Assign the first new file detected to spectrum_filename
            spectrum_filename = new_files[0]  # This assigns the new file added to the folder
            print(f"New file detected: {spectrum_filename}")

            try:
                    # Attempt to read the new spectrum file into a DataFrame
                spectrum_filepath = os.path.join(folder_path, spectrum_filename)
                new_spectrum_df = pd.read_csv(spectrum_filepath, header=None, skiprows = 14, delimiter='\t')
                new_spectrum_df = process_spectrum(new_spectrum_df)
                    # Process the new spectrum file (if required, use your existing process_spectrum function here)
                print(new_spectrum_df.head())  # Print the first few rows to verify it's loaded correctly
                processed_files.add(spectrum_filename)  # Add this file to the processed list
                break  # Exit the loop after processing the new file

            except Exception as e:
                print(f"Error reading file {spectrum_filename}: {e}")

        time.sleep(1)  # Check every second for new files

        # Now you have the processed DataFrame 'new_spectrum_df'

    # Calculate the result
    result = objective_function(new_spectrum_df)
    peak_position_values.append(result[obj1_name])
    intensity_ratio_values.append(result[obj2_name])

    ax_client.complete_trial(trial_index=trial_index, raw_data={
        obj1_name: result["peak_position"], 
        obj2_name: result["intensity_ratio"]
    })
    print('Experiment trial complete...')
    print('Begin pump cleaning...')

    write_suggested_parameters_2(0,0,6000,5000)
    print('Cleaning second time...')
    write_suggested_parameters_2(0,0,6000,5000)
    print('Cleaning third time...')
    write_suggested_parameters_2(0,0,6000,5000)
    print('Making the reactor ready for next iteration...')
    write_suggested_parameters_2(0,0,6000,0)

    update_plots(fig, axs, ref_df, new_spectrum_df, iteration_count + 1, peak_position_values, intensity_ratio_values, parameter_history)
    iteration_count += 1

plt.ioff()  # Turn off interactive mode
plt.show()

print("All iterations completed.")

df_res = ax_client.get_trials_data_frame()
print(Fore.GREEN + 'Results',  df_res)
df_res.to_csv(DATA_UV_DIR_PATH + 'parameters_objective_result.csv', index=False)

from ax.utils.notebook.plotting import init_notebook_plotting, render
from ax.plot.contour import plot_contour

import plotly.io as pio
pio.renderers.default = "browser"

model = ax_client.generation_strategy.model
render(ax_client.interact_contour(model= model, metric_name="peak_position"))
render(ax_client.interact_contour(model= model, metric_name="intensity_ratio"))

SyntaxError: invalid syntax (1189400457.py, line 157)

In [3]:
baseline_data = pd.read_csv('spectrometer_data.txt', delimiter='\t')
baseline_data

,wavelength,dark,baseline
0,198.913910,3542.0,3541.0
1,199.384414,2588.0,2592.0
2,199.854890,2755.0,2757.0
3,200.325337,2721.0,2722.0
4,200.795756,2769.0,2786.0
...,...,...,...
2063,1109.412141,2753.0,2909.0
2064,1109.824415,2721.0,2723.0
2065,1110.236661,2722.0,2728.0
2066,1110.648879,2725.0,2728.0
